# GRASS Tool with Parameters

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# %end
# %option G_OPT_M_COORDS
# %end
# %option G_OPT_R_INPUTS
# %end
# %option G_OPT_F_OUTPUT
# % guisection: Output
# %end
# %option
# % key: format
# % type: string
# % required: yes
# % options: json,csv
# % label: Output format
# % descriptions: json;JSON (JavaScript Object Notation);csv;CSV (Comma Separated Values)
# % answer: json
# % guisection: Output
# %end


import atexit
import subprocess
import sys
import csv
import json
import io

import grass.script as gs


def output_results(results, sample_rasters, file_name, file_format):
    if file_format == "json":
        with open(file_name, "w", encoding="utf-8") as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)
    elif file_format == "csv":
        with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
            header = ["name"]
            header.extend(results[sample_rasters[0]].keys())
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            for key, value in results.items():
                row = {"name": key}
                row.update(value)
                writer.writerow(row)
    else:
        raise ValueError(f"Unsupported or invalid format: {file_format}")


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def viewshed(
    elevation,
    coordinates,
    sample_rasters,
    output,
    file_format,
):
    viewshed = gs.append_node_pid("tmp_viewshed")
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        # While we could use .strip().splitlines()[-1].split(",") here,
        # using a proper CSV reader is more robust.
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    output_results(
        results=results,
        sample_rasters=sample_rasters,
        file_name=output,
        file_format=file_format,
    )


def main():
    options, flags = gs.parser()
    coordinates = options["coordinates"].split(",")
    sample_rasters = options["input"].split(",")
    viewshed(
        elevation=options["elevation"],
        coordinates=(float(coordinates[0]), float(coordinates[1])),
        sample_rasters=sample_rasters,
        output=options["output"],
        file_format=options["format"],
    )


if __name__ == "__main__":
    main()

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./viewscape.py --interface-description

Running the script with `--html-description` gives the command line interface described in HTML which later becomes a part of the tool's HTML documentation:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py --html-description > test.html

In [ ]:
from IPython.display import IFrame

IFrame("test.html", width=700, height=600)

Open the generated HTML file called _test.html_ from the File Browser (on the left in JupyterLab).

On desktop, a graphical user interface for the tool would be available, too, accessible, e.g., through `--ui`:

```bash
grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py --ui
```

The GUI window may look like this:

![GUI with vector and raster parameters](img/vector_raster_gui.png)

## Using the Parameters

The values parsed from the command line are stored in a dictionary returned by the _parse_ function. They can be accessed using `dictionary["name"]` syntax where _name_ is the name of the parameter. We are using the predefined standard options for vector input and raster output which are named _input_ and _output_. Here, we add an additional parameter named _layer_ also referred to as _field_ which can be used to specify a layer or a subset in the input vector dataset (more on that later).

We will store the three values in Python variables and pass them to _v.to.rast_ which will do the actual processing for us.

In [ ]:
%%writefile vector_to_raster.py
#!/usr/bin/env python

# %module
# % description: Converts vector data to raster data
# %end
# %option G_OPT_V_INPUT
# %end
# %option G_OPT_V_FIELD
# %end
# %option G_OPT_R_OUTPUT
# %end

import subprocess
import sys

import grass.script as gs


def main():
    options, flags = gs.parser()
    vector_input = options["input"]
    vector_layer = options["layer"]
    raster_output = options["output"]

    gs.run_command(
        "v.to.rast",
        input=vector_input,
        layer=vector_layer,
        output=raster_output,
        use="val",
    )


if __name__ == "__main__":
    main()

We can now execute the tool. There is no need to make it executable again because we are using the same name as before and the execute permissions are preserved even when the file contents change.

The command line parameters in GRASS GIS are key-value pairs which are using syntax `key=value`. In the CLI world, this is sometimes called _named arguments_ and it is similar to Python keyword arguments.

The dataset we are using here has vector points called _firestations_ and we will create new raster called _stations_. Before running the processing, we will set the computation region to the extent of _firestations_ and will use resolution 30 meters (this is set and preserved for the mapset between the individual runs of `grass ... --exec`).

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec g.region vector=firestations res=100
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py input=firestations output=stations

Try running the above again. The raster named _stations_ now exists, so GRASS GIS will automatically detect that and ask you to use `--overwrite` if you want to replace the existing data.

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py input=firestations output=stations

With added `--overwrite`:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py input=firestations output=stations --overwrite

Let's view data range of the newly created raster:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.info map=stations -r

To view the data in the notebook we will render the raster using _grass.jupyter.Map_. Usually, we would just create a GRASS session or already have one. However, to keep our development environment as is, we will avoid creating a session in the notebook process, but using a subprocess to do the rendering into a PNG image. The following uses the `%%python` magic to execute Python code in a subprocess:

In [ ]:
%%python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.jupyter as gj
import grass.script.setup  # Needed only in 8.2 and older.

with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
    ortho_map = gj.Map()
    ortho_map.d_rast(map="stations")
    # Save the image (in a standard notebook, we would just display the image now).
    ortho_map.save("stations.png")

Now, use _Image_ from _IPython.display_ to display the PNG:

In [ ]:
from IPython.display import Image

Image("stations.png")

## General Parameter Definition

To add general parameters such as text and numbers, we can use the following key-value syntax enclosed in `%option` and `%end`:

```python
# %option
# % key1: value1
# % key2: value2
# % key3: value3
# %end
```

Let's say we want to allow users of our tool to specify the raster value which is used where vector features are present. We will name it _value_ (`key: value`) and make it required (`required: yes`). The data type we will use is _double_ (`type: double`) which we can use as _float_ in Python. The following puts all these together:

```python
# %option
# % key: value
# % type: double
# % required: yes
# % description: Raster cell value where features are
# %end
```

Here is the full script. The values come as strings, so for raster value, we convert the string to float (`float(options["value"])`), although in this case we don't have to because we just pass it to the _v.to.rast_ subprocess and a string would work in that context too.

In [ ]:
%%writefile vector_to_raster.py
#!/usr/bin/env python

# %module
# % description: Converts vector data to raster data
# %end
# %option G_OPT_V_INPUT
# %end
# %option G_OPT_V_FIELD
# %end
# %option G_OPT_R_OUTPUT
# %end
# %option
# % key: value
# % type: double
# % required: yes
# % description: Raster cell value where features are
# %end

import subprocess
import sys

import grass.script as gs


def main():
    options, flags = gs.parser()
    vector_input = options["input"]
    vector_layer = options["layer"]
    raster_output = options["output"]
    value = float(options["value"])

    gs.run_command(
        "v.to.rast",
        input=vector_input,
        layer=vector_layer,
        output=raster_output,
        use="val",
        value=value,
    )


if __name__ == "__main__":
    main()

Now, let's add the new parameter `value=5`:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec ./vector_to_raster.py input=firestations output=stations_value value=5

The data range the raster is now 5-5, i.e., all values are 5:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.info map=stations_value -r

## Using the New Tool from Python

The tool can be used from Python just like the other GRASS tools.

Here is a Python script which creates a GRASS session and calls our new tool:

In [ ]:
%%python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.script.setup


def main():
    with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
        gs.run_command(
            "vector_to_raster.py",
            input="firestations",
            output="stations_python",
            value=5,
        )


if __name__ == "__main__":
    main()

## Using Existing Interfaces for Generating Wrappers and Boilerplates

Often, a new tool is somehow wrapping or extending an existing tool or is similar to one. To quickly generate a boilerplate code in such cases, we can run any GRASS tool with `--script`. Unfortunately, `--script` does not currently output standard options, so the generated definitions are unnecessarily complicated.

Given that the same structure is needed every time, it is a good idea to use `--script` or copy-paste code from existing tools or examples. In the GRASS GIS source code, the Python scripts are under _[scripts](https://github.com/OSGeo/grass/tree/releasebranch_8_2/scripts)_ and _[temporal](https://github.com/OSGeo/grass/tree/releasebranch_8_2/temporal)_. Tools in the grass-addons repository are not organized by language, but many of the tools are in Python.

Here is how to get a Python script boilerplate from _v.to.rast_ (which itself is in written C):

In [ ]:
!grass --tmp-location XY --exec v.to.rast --script